In [1]:
import sys
import numpy as np
sys.path.insert(0, '/home/ghn8/lib/ugscnn/meshcnn')
from mesh_utils import *
import scipy.sparse as sparse
sys.path.insert(0, '/nfs03/users/ghn8/TaskContrastsFromRest/libigl/python')
import pyigl as igl
import pickle
import os
import nibabel.gifti as gi



In [2]:
# from https://github.com/maxjiang93/ugscnn

class icosphere(object):
    def __init__(self, vertices, faces, level=0, nv_prev=0, nv_next=0):
        self.level = level
        self.vertices, self.faces = vertices, faces
        self.intp = None
        self.v0, self.f0 = self.vertices.copy(), self.faces.copy()

        self.normalize()
        self.lat, self.long = self.xyz2latlong()
        self.nf, self.nv = self.faces.shape[0], self.vertices.shape[0]

        self.nv_prev = nv_prev
        self.nv_next = nv_next

        self.construct_matrices()
        self.info = {"V": self.vertices,
                     "F": self.faces,
                     "nv_prev": self.nv_prev,
                     "nv_next": self.nv_next,
                     "G": self.G,
                     "L": self.L,
                     "N": self.N,
                     "NS": self.NS,
                     "EW": self.EW,
                     "F2V": self.F2V,
                     "M": self.M,
                     "Seq": self.Seq,
                     "Intp": self.Intp}
        
    def normalize(self, radius=1):
        '''
        Reproject to spherical surface
        '''
        vectors = self.vertices
        scalar = (vectors ** 2).sum(axis=1)**.5
        unit = vectors / scalar.reshape((-1, 1))
        offset = radius - scalar
        self.vertices = self.vertices + unit * offset.reshape((-1, 1))
        
    def xyz2latlong(self):
        x, y, z = self.vertices[:, 0], self.vertices[:, 1], self.vertices[:, 2]
        long = np.arctan2(y, x)
        xy2 = x**2 + y**2
        lat = np.arctan2(z, np.sqrt(xy2))
        return lat, long
    
    def construct_matrices(self):
        """
        Construct FEM matrices
        """
        V = p2e(self.vertices)
        F = p2e(self.faces)
        # Compute gradient operator: #F*3 by #V
        G = igl.eigen.SparseMatrixd()
        L = igl.eigen.SparseMatrixd()
        M = igl.eigen.SparseMatrixd()
        N = igl.eigen.MatrixXd()
        A = igl.eigen.MatrixXd()
        igl.grad(V, F, G)
        igl.cotmatrix(V, F, L)
        igl.per_face_normals(V, F, N)
        igl.doublearea(V, F, A)
        igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI, M)
        G = e2p(G)
        L = e2p(L)
        N = e2p(N)
        A = e2p(A)
        M = e2p(M)
        M = M.data
        # Compute latitude and longitude directional vector fields
        NS = np.reshape(G.dot(self.lat), [self.nf, 3], order='F')
        EW = np.cross(NS, N)
        # Compute F2V matrix (weigh by area)
        # adjacency
        i = self.faces.ravel()
        j = np.arange(self.nf).repeat(3)
        one = np.ones(self.nf * 3)
        adj = sparse.csc_matrix((one, (i, j)), shape=(self.nv, self.nf))
        tot_area = adj.dot(A)
        norm_area = A.ravel().repeat(3)/np.squeeze(tot_area[i])
        F2V = sparse.csc_matrix((norm_area, (i, j)), shape=(self.nv, self.nf))
        # Compute interpolation matrix
        if self.level > 0:
            intp = self.intp[self.nv_prev:]
            i = np.concatenate((np.arange(self.nv), np.arange(self.nv_prev, self.nv)))
            j = np.concatenate((np.arange(self.nv_prev), intp[:, 0], intp[:, 1]))
            ratio = np.concatenate((np.ones(self.nv_prev), 0.5*np.ones(2*intp.shape[0])))
            intp = sparse.csc_matrix((ratio, (i, j)), shape=(self.nv, self.nv_prev))
        else:
            intp = sparse.csc_matrix(np.eye(self.nv))


        # Compute vertex mean matrix
        self.G = G  # gradient matrix
        self.L = L  # laplacian matrix
        self.N = N  # normal vectors (per-triangle)
        self.NS = NS  # north-south vectors (per-triangle)
        self.EW = EW  # east-west vectors (per-triangle)
        self.F2V = F2V  # map face quantities to vertices
        self.M = M # mass matrix (area of voronoi cell around node. for integration)
        self.Seq = self._rotseq(self.vertices)
        self.Intp = intp

    def _find_neighbor(self, F, ind):
        """find a icosahedron neighbor of vertex i"""
        FF = [F[i] for i in range(F.shape[0]) if ind in F[i]]
        FF = np.concatenate(FF)
        FF = np.unique(FF)
        neigh = [f for f in FF if f != ind]
        return neigh
    
    def _rot_matrix(self, rot_axis, cos_t, sin_t):
        k = rot_axis / np.linalg.norm(rot_axis)
        I = np.eye(3)

        R = []
        for i in range(3):
            v = I[i]
            vr = v*cos_t+np.cross(k, v)*sin_t+k*(k.dot(v))*(1-cos_t)
            R.append(vr)
        R = np.stack(R, axis=-1)
        return R

    def _ico_rot_matrix(self, ind):
        """
        return rotation matrix to perform permutation corresponding to 
        moving a certain icosahedron node to the top
        """
        v0_ = self.v0.copy()
        f0_ = self.f0.copy()
        V0 = v0_[ind]
        Z0 = np.array([0, 0, 1])

        # rotate the point to the top (+z)
        k = np.cross(V0, Z0)
        ct = np.dot(V0, Z0)
        st = -np.linalg.norm(k)
        R = self._rot_matrix(k, ct, st)
        v0_ = v0_.dot(R)

        # rotate a neighbor to align with (+y)
        ni = self._find_neighbor(f0_, ind)[0]
        vec = v0_[ni].copy()
        vec[2] = 0
        vec = vec/np.linalg.norm(vec)
        y_ = np.eye(3)[1]

        k = np.eye(3)[2]
        crs = np.cross(vec, y_)
        ct = np.dot(vec, y_)
        st = -np.sign(crs[-1])*np.linalg.norm(crs)

        R2 = self._rot_matrix(k, ct, st)
        return R.dot(R2)

    def _rotseq(self, V, acc=9):
        """sequence to move an original node on icosahedron to top"""
        seq = []
        for i in range(11):
            Vr = V.dot(self._ico_rot_matrix(i))
            # lexsort
            s1 = np.lexsort(np.round(V.T, acc))
            s2 = np.lexsort(np.round(Vr.T, acc))
            s = s1[np.argsort(s2)]
            seq.append(s)
        return tuple(seq)
    
    def export_mesh_info(self, filename):
        """Write mesh info as pickle file"""
        with open(filename, "wb") as f:
            pickle.dump(self.info, f)

        

In [3]:
def get_corresponding_vertices_between_icospheres(ico1_vertices, ico2_vertices):
    closest_vertices = []
    closest_dists = []
    for i in range(ico1_vertices.shape[0]):
        if i % 1000 == 0:
            print(i, "/", ico1_vertices.shape[0])
        ico1_v = ico1_vertices[i, :]
        
        min_dist = sys.float_info.max
        closest_point = -1
        for j in range(ico2_vertices.shape[0]):
            ico2_v = ico2_vertices[j, :]
            dist = np.mean((ico1_v - ico2_v)**2)
            
            if dist == 0:
                min_dist = 0
                closest_point = j
                break
            elif dist < min_dist:
                min_dist = dist
                closest_point = j
        closest_vertices.append(closest_point)
        closest_dists.append(min_dist)

    return closest_vertices, closest_dists

In [4]:
def get_nearest_vertices_in_lower_res_icosphere(highres_ico_vertices, lowres_ico_vertices, num_neighbors=7):
    closest_vertices = []
    closest_dists = []
    for i in range(highres_ico_vertices.shape[0]):
        if i % 1000 == 0:
            print(i, "/", highres_ico_vertices.shape[0])
        ico1_v = highres_ico_vertices[i, :]

        dists = np.mean((lowres_ico_vertices - ico1_v)**2, axis=1)
        sorted_dist_indices = np.argsort(dists)
        nearest_indices = sorted_dist_indices[:num_neighbors]
        nearest_dists = dists[nearest_indices]
        
        closest_vertices.append(nearest_indices)
        closest_dists.append(nearest_dists)

                
    return np.asarray(closest_vertices), np.asarray(closest_dists)

In [5]:
# 

mesh50 = gi.read ('/nfs03/data/HCPpipelines/global/templates/standard_mesh_atlases/L.sphere.50_fs_LR.surf.gii')
mesh100 = gi.read ('/nfs03/data/HCPpipelines/global/templates/standard_mesh_atlases/L.sphere.100_fs_LR.surf.gii')
mesh200 = gi.read ('/nfs03/data/HCPpipelines/global/templates/standard_mesh_atlases/L.sphere.200_fs_LR.surf.gii')


/home/ghn8/.conda/envs/brain_surf_cnn/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: giftiio.read function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  """Entry point for launching an IPython kernel.
/home/ghn8/.conda/envs/brain_surf_cnn/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: giftiio.read function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  
/home/ghn8/.conda/envs/brain_surf_cnn/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: giftiio.read function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  This is separate from the ipykernel package so we can avoid doing imports u

In [6]:
ico50 = icosphere(mesh50.darrays[0].data, mesh50.darrays[1].data, level=0, nv_prev=0, nv_next=len(mesh100.darrays[0].data))
ico100 = icosphere(mesh100.darrays[0].data, mesh100.darrays[1].data, level=0, nv_prev=len(mesh50.darrays[0].data), nv_next=len(mesh200.darrays[0].data))
ico200 = icosphere(mesh200.darrays[0].data, mesh200.darrays[1].data, level=0, nv_prev=len(mesh100.darrays[0].data), nv_next=len(mesh500.darrays[0].data))
ico500 = icosphere(mesh500.darrays[0].data, mesh500.darrays[1].data, level=0, nv_prev=len(mesh200.darrays[0].data), nv_next=len(mesh1k.darrays[0].data))
ico1k = icosphere(mesh1k.darrays[0].data, mesh1k.darrays[1].data, level=0, nv_prev=len(mesh500.darrays[0].data), nv_next=len(mesh2k.darrays[0].data))
ico2k = icosphere(mesh2k.darrays[0].data, mesh2k.darrays[1].data, level=0, nv_prev=len(mesh1k.darrays[0].data), nv_next=len(mesh10k.darrays[0].data))
ico2k = icosphere(mesh2k.darrays[0].data, mesh2k.darrays[1].data, level=0, nv_prev=len(mesh1k.darrays[0].data), nv_next=len(mesh10k.darrays[0].data))
ico10k = icosphere(mesh10k.darrays[0].data, mesh10k.darrays[1].data, level=0, nv_prev=len(mesh2k.darrays[0].data), nv_next=len(mesh32k.darrays[0].data))
ico32k = icosphere(mesh32k.darrays[0].data, mesh32k.darrays[1].data, level=0, nv_prev=len(mesh10k.darrays[0].data), nv_next=0)



In [7]:
print(ico50.nv, ico50.nv_prev, ico50.nv_next)
print(ico100.nv, ico100.nv_prev, ico100.nv_next)
print(ico200.nv, ico200.nv_prev, ico200.nv_next)
print(ico500.nv, ico500.nv_prev, ico500.nv_next)
print(ico1k.nv, ico1k.nv_prev, ico1k.nv_next)
print(ico2k.nv, ico2k.nv_prev, ico2k.nv_next)
print(ico10k.nv, ico10k.nv_prev, ico10k.nv_next)
print(ico32k.nv, ico32k.nv_prev, ico32k.nv_next)

42 0 92
92 42 162
162 92 492
492 162 1002
1002 492 2562
2562 1002 10242
10242 2562 32492
32492 10242 0


# 1k - 4k - 32k

In [12]:
ico1k = icosphere(mesh1k.darrays[0].data, mesh1k.darrays[1].data, level=0, nv_prev=0, nv_next=len(mesh4k.darrays[0].data))
ico4k = icosphere(mesh4k.darrays[0].data, mesh4k.darrays[1].data, level=0, nv_prev=len(mesh1k.darrays[0].data), nv_next=len(mesh32k.darrays[0].data))
ico32k = icosphere(mesh32k.darrays[0].data, mesh32k.darrays[1].data, level=0, nv_prev=len(mesh4k.darrays[0].data), nv_next=0)



In [14]:
ico1k.export_mesh_info('meshes/1k-4k-32k/ico1k.pkl')
ico4k.export_mesh_info('meshes/1k-4k-32k/ico4k.pkl')
ico32k.export_mesh_info('meshes/1k-4k-32k/ico32k.pkl')

In [15]:
all_icos = [ico1k, ico4k, ico32k]
all_ico_names = ["icosphere_0", "icosphere_1", "icosphere_2"]

for ico in all_icos:
    print(ico.nv, ico.nv_prev, ico.nv_next)

1002 0 4002
4002 1002 32492
32492 4002 0


In [16]:
for i in range(len(all_icos)):
    ico = all_icos[i]
    name = all_ico_names[i]
    if not os.path.exists("meshes/1k-4k-32k/%s_neighbor_patches.npy" % name):
        ico_neighbors = []
        for i in range(len(ico.vertices)):
            neighbors = ico._find_neighbor(ico.faces, i)
            patch = [i] + neighbors
            if len(patch) < 7:
                patch = [i] + patch
            ico_neighbors.append(np.asarray(patch))
        ico_neighbors = np.asarray(ico_neighbors)
        print(ico_neighbors.shape)
        np.save("meshes/1k-4k-32k/%s_neighbor_patches.npy" % name, ico_neighbors)

(1002, 7)
(4002, 7)
(32492, 7)


In [18]:
for i in range(len(all_icos)-1, 0, -1):
    highres_ico = all_icos[i]
    highres_name =all_ico_names[i]
    lowres_ico = all_icos[i-1]
    lowres_name =all_ico_names[i-1]
    
    if not os.path.exists('meshes/1k-4k-32k/%s_to_%s_vertices.npy' % (highres_name, lowres_name)):
        nearest_vertices, nearest_dists = get_nearest_vertices_in_lower_res_icosphere(highres_ico.vertices, lowres_ico.vertices)

        np.save('meshes/1k-4k-32k/%s_to_%s_vertices.npy' % (highres_name, lowres_name), nearest_vertices)
        np.save('meshes/1k-4k-32k/%s_to_%s_vertices_closests_dists.npy' % (highres_name, lowres_name), nearest_dists)



In [19]:
for i in range(len(all_icos)-1):
    highres_ico = all_icos[i+1]
    highres_name =all_ico_names[i+1]
    lowres_ico = all_icos[i]
    lowres_name =all_ico_names[i]
    
    if not os.path.exists('meshes/1k-4k-32k/%s_to_%s_vertices.npy' % (lowres_name, highres_name)):
        nearest_vertices, nearest_dists = get_corresponding_vertices_between_icospheres(lowres_ico.vertices, highres_ico.vertices)

        print("%s_to_%s" % (lowres_name, highres_name))
        np.save('meshes/1k-4k-32k/%s_to_%s_vertices.npy' % (lowres_name, highres_name), nearest_vertices)
        np.save('meshes/1k-4k-32k/%s_to_%s_vertices_closests_dists.npy' % (lowres_name, highres_name), nearest_dists)



0 / 1002
1000 / 1002
icosphere_0_to_icosphere_1
0 / 4002
1000 / 4002
2000 / 4002
3000 / 4002
4000 / 4002
icosphere_1_to_icosphere_2


# 500 - 2k - 8k - 32k

In [8]:
ico500 = icosphere(mesh500.darrays[0].data, mesh500.darrays[1].data, level=0, nv_prev=0, nv_next=len(mesh2k.darrays[0].data))
ico2k = icosphere(mesh2k.darrays[0].data, mesh2k.darrays[1].data, level=0, nv_prev=len(mesh500.darrays[0].data), nv_next=len(mesh2k.darrays[0].data))
ico8k = icosphere(mesh8k.darrays[0].data, mesh8k.darrays[1].data, level=0, nv_prev=len(mesh2k.darrays[0].data), nv_next=len(mesh32k.darrays[0].data))
ico32k = icosphere(mesh32k.darrays[0].data, mesh32k.darrays[1].data, level=0, nv_prev=len(mesh8k.darrays[0].data), nv_next=0)



In [9]:
all_icos = [ico500, ico2k, ico8k, ico32k]
all_ico_names = ["ico500", "ico2k", "ico8k", "ico32k"]

for ico in all_icos:
    print(ico.nv, ico.nv_prev, ico.nv_next)

492 0 2562
2562 492 2562
7842 2562 32492
32492 7842 0


In [14]:
ico500.export_mesh_info('meshes/500-2k-8k-32k/icosphere_500.pkl')

In [11]:
for i in range(len(all_icos)):
    ico = all_icos[i]
    name = all_ico_names[i]
    if not os.path.exists("meshes/%s_neighbor_patches.npy" % name):
        ico_neighbors = []
        for i in range(len(ico.vertices)):
            neighbors = ico._find_neighbor(ico.faces, i)
            patch = [i] + neighbors
            if len(patch) < 7:
                patch = [i] + patch
            ico_neighbors.append(np.asarray(patch))
        ico_neighbors = np.asarray(ico_neighbors)
        print(ico_neighbors.shape)
        np.save("meshes/%s_neighbor_patches.npy" % name, ico_neighbors)

In [12]:
for i in range(len(all_icos)-1, 0, -1):
    highres_ico = all_icos[i]
    highres_name =all_ico_names[i]
    lowres_ico = all_icos[i-1]
    lowres_name =all_ico_names[i-1]
    
    if not os.path.exists('meshes/%s_to_%s_vertices.npy' % (highres_name, lowres_name)):
        nearest_vertices, nearest_dists = get_nearest_vertices_in_lower_res_icosphere(highres_ico.vertices, lowres_ico.vertices)

        np.save('meshes/%s_to_%s_vertices.npy' % (highres_name, lowres_name), nearest_vertices)
        np.save('meshes/%s_to_%s_vertices_closests_dists.npy' % (highres_name, lowres_name), nearest_dists)



0 / 32492
1000 / 32492
2000 / 32492
3000 / 32492
4000 / 32492
5000 / 32492
6000 / 32492
7000 / 32492
8000 / 32492
9000 / 32492
10000 / 32492
11000 / 32492
12000 / 32492
13000 / 32492
14000 / 32492
15000 / 32492
16000 / 32492
17000 / 32492
18000 / 32492
19000 / 32492
20000 / 32492
21000 / 32492
22000 / 32492
23000 / 32492
24000 / 32492
25000 / 32492
26000 / 32492
27000 / 32492
28000 / 32492
29000 / 32492
30000 / 32492
31000 / 32492
32000 / 32492
0 / 7842
1000 / 7842
2000 / 7842
3000 / 7842
4000 / 7842
5000 / 7842
6000 / 7842
7000 / 7842
0 / 2562
1000 / 2562
2000 / 2562


In [13]:
for i in range(len(all_icos)-1):
    highres_ico = all_icos[i+1]
    highres_name =all_ico_names[i+1]
    lowres_ico = all_icos[i]
    lowres_name =all_ico_names[i]
    
    if not os.path.exists('meshes/%s_to_%s_vertices.npy' % (lowres_name, highres_name)):
        nearest_vertices, nearest_dists = get_corresponding_vertices_between_icospheres(lowres_ico.vertices, highres_ico.vertices)

        print("%s_to_%s" % (lowres_name, highres_name))
        np.save('meshes/%s_to_%s_vertices.npy' % (lowres_name, highres_name), nearest_vertices)
        np.save('meshes/%s_to_%s_vertices_closests_dists.npy' % (lowres_name, highres_name), nearest_dists)



0 / 492
ico500_to_ico2k
0 / 2562
1000 / 2562
2000 / 2562
ico2k_to_ico8k
0 / 7842
1000 / 7842
2000 / 7842
3000 / 7842
4000 / 7842
5000 / 7842
6000 / 7842
7000 / 7842
ico8k_to_ico32k


# 100-500-2k-8k-32k

In [15]:
ico100 = icosphere(mesh100.darrays[0].data, mesh100.darrays[1].data, level=0, nv_prev=0, nv_next=len(mesh500.darrays[0].data))
ico500 = icosphere(mesh500.darrays[0].data, mesh500.darrays[1].data, level=0, nv_prev=len(mesh100.darrays[0].data), nv_next=len(mesh2k.darrays[0].data))
ico2k = icosphere(mesh2k.darrays[0].data, mesh2k.darrays[1].data, level=0, nv_prev=len(mesh500.darrays[0].data), nv_next=len(mesh2k.darrays[0].data))
ico8k = icosphere(mesh8k.darrays[0].data, mesh8k.darrays[1].data, level=0, nv_prev=len(mesh2k.darrays[0].data), nv_next=len(mesh32k.darrays[0].data))
ico32k = icosphere(mesh32k.darrays[0].data, mesh32k.darrays[1].data, level=0, nv_prev=len(mesh8k.darrays[0].data), nv_next=0)


In [18]:
output_dir = "100-500-2k-8k-32k"
ico100.export_mesh_info(os.path.join("meshes", output_dir, "icosphere_100.pkl"))
ico500.export_mesh_info(os.path.join("meshes", output_dir, "icosphere_500.pkl"))


In [17]:
all_icos = [ico100, ico500, ico2k, ico8k, ico32k]
all_ico_names = ["ico100", "ico500", "ico2k", "ico8k", "ico32k"]

for ico in all_icos:
    print(ico.nv, ico.nv_prev, ico.nv_next)

92 0 492
492 92 2562
2562 492 2562
7842 2562 32492
32492 7842 0


In [20]:
for i in range(len(all_icos)):
    ico = all_icos[i]
    name = all_ico_names[i]
    if not os.path.exists(os.path.join("meshes", output_dir, "%s_neighbor_patches.npy" % name)):
        ico_neighbors = []
        for i in range(len(ico.vertices)):
            neighbors = ico._find_neighbor(ico.faces, i)
            patch = [i] + neighbors
            if len(patch) < 7:
                patch = [i] + patch
            ico_neighbors.append(np.asarray(patch))
        ico_neighbors = np.asarray(ico_neighbors)
        print(ico_neighbors.shape)
        np.save(os.path.join("meshes", output_dir, "%s_neighbor_patches.npy" % name), ico_neighbors)

(92, 7)


In [22]:
for i in range(len(all_icos)-1):
    highres_ico = all_icos[i+1]
    highres_name =all_ico_names[i+1]
    lowres_ico = all_icos[i]
    lowres_name =all_ico_names[i]
    
    if not os.path.exists(os.path.join("meshes", output_dir, "%s_to_%s_vertices.npy" % (lowres_name, highres_name))):
        nearest_vertices, nearest_dists = get_corresponding_vertices_between_icospheres(lowres_ico.vertices, highres_ico.vertices)

        print("%s_to_%s" % (lowres_name, highres_name))
        np.save(os.path.join("meshes", output_dir, "%s_to_%s_vertices.npy" % (lowres_name, highres_name)), nearest_vertices)
        np.save(os.path.join("meshes", output_dir, "%s_to_%s_vertices_closests_dists.npy" % (lowres_name, highres_name)), nearest_dists)


0 / 92
ico100_to_ico500


In [28]:
for i in range(len(all_icos)-1, 0, -1):
    highres_ico = all_icos[i]
    highres_name = all_ico_names[i]
    lowres_ico = all_icos[i-1]
    lowres_name = all_ico_names[i-1]
    
    if not os.path.exists(os.path.join("meshes", output_dir, "%s_to_%s_vertices.npy" % (highres_name, lowres_name))):
        print("%s_to_%s" % (highres_name, lowres_name))
        nearest_vertices, nearest_dists = get_nearest_vertices_in_lower_res_icosphere(highres_ico.vertices, lowres_ico.vertices)

        np.save(os.path.join("meshes", output_dir, '%s_to_%s_vertices.npy' % (highres_name, lowres_name)), nearest_vertices)
        np.save(os.path.join("meshes", output_dir, '%s_to_%s_vertices_closests_dists.npy' % (highres_name, lowres_name)), nearest_dists)



ico500_to_ico100
0 / 492


In [25]:
all_ico_names

['ico100', 'ico500', 'ico2k', 'ico8k', 'ico32k']

# 4 levels: 2k - 4k - 8k - 16k - 32k

In [23]:
ico2k = icosphere(mesh2k.darrays[0].data, mesh2k.darrays[1].data, level=0, nv_prev=0, nv_next=len(mesh4k.darrays[0].data))
ico4k = icosphere(mesh4k.darrays[0].data, mesh4k.darrays[1].data, level=0, nv_prev=len(mesh2k.darrays[0].data), nv_next=len(mesh8k.darrays[0].data))
ico8k = icosphere(mesh8k.darrays[0].data, mesh8k.darrays[1].data, level=0, nv_prev=len(mesh4k.darrays[0].data), nv_next=len(mesh16k.darrays[0].data))
ico16k = icosphere(mesh16k.darrays[0].data, mesh16k.darrays[1].data, level=0, nv_prev=len(mesh8k.darrays[0].data), nv_next=len(mesh32k.darrays[0].data))
ico32k = icosphere(mesh32k.darrays[0].data, mesh32k.darrays[1].data, level=0, nv_prev=len(mesh16k.darrays[0].data), nv_next=0)



In [31]:
ico50.export_mesh_info('meshes/icosphere_50.pkl')
ico100.export_mesh_info('meshes/icosphere_100.pkl')
ico200.export_mesh_info('meshes/icosphere_200.pkl')
ico500.export_mesh_info('meshes/icosphere_500.pkl')
ico1k.export_mesh_info('meshes/icosphere_1k.pkl')

In [27]:
# all_icos = [ico2k, ico4k, ico8k, ico16k, ico32k]
# all_ico_names = ["ico2k", "ico4k", "ico8k", "ico16k", "ico32k"]

all_icos = [ico50, ico100, ico200, ico500, ico1k, ico2k]
all_ico_names = ["ico50", "ico100", "ico200", "ico500", "ico1k", "ico2k"]

for ico in all_icos:
    print(ico.nv, ico.nv_prev, ico.nv_next)

42 0 92
92 42 162
162 92 492
492 162 1002
1002 492 2562
2562 0 4002


## Neighbors

In [28]:
for i in range(len(all_icos)):
    ico = all_icos[i]
    name = all_ico_names[i]
    ico_neighbors = []
    for i in range(len(ico.vertices)):
        neighbors = ico._find_neighbor(ico.faces, i)
        patch = [i] + neighbors
        if len(patch) < 7:
            patch = [i] + patch
        ico_neighbors.append(np.asarray(patch))
    ico_neighbors = np.asarray(ico_neighbors)
    print(ico_neighbors.shape)
    np.save("meshes/%s_neighbor_patches.npy" % name, ico_neighbors)

(42, 7)
(92, 7)
(162, 7)
(492, 7)
(1002, 7)
(2562, 7)


## High-rest to low-res mapping

In [29]:
for i in range(len(all_icos)-1, 0, -1):
    highres_ico = all_icos[i]
    highres_name =all_ico_names[i]
    lowres_ico = all_icos[i-1]
    lowres_name =all_ico_names[i-1]
    
    nearest_vertices, nearest_dists = get_nearest_vertices_in_lower_res_icosphere(highres_ico.vertices, lowres_ico.vertices)
    
    np.save('meshes/%s_to_%s_vertices.npy' % (highres_name, lowres_name), nearest_vertices)
    np.save('meshes/%s_to_%s_vertices_closests_dists.npy' % (highres_name, lowres_name), nearest_dists)



0 / 2562
1000 / 2562
2000 / 2562
0 / 1002
1000 / 1002
0 / 492
0 / 162
0 / 92


## Low-res to high-res mapping

In [30]:
for i in range(len(all_icos)-1):
    highres_ico = all_icos[i+1]
    highres_name =all_ico_names[i+1]
    lowres_ico = all_icos[i]
    lowres_name =all_ico_names[i]
    
    nearest_vertices, nearest_dists = get_corresponding_vertices_between_icospheres(lowres_ico.vertices, highres_ico.vertices)
    
    print("%s_to_%s" % (lowres_name, highres_name))
    np.save('meshes/%s_to_%s_vertices.npy' % (lowres_name, highres_name), nearest_vertices)
    np.save('meshes/%s_to_%s_vertices_closests_dists.npy' % (lowres_name, highres_name), nearest_dists)



0 / 42
ico50_to_ico100
0 / 92
ico100_to_ico200
0 / 162
ico200_to_ico500
0 / 492
ico500_to_ico1k
0 / 1002
1000 / 1002
ico1k_to_ico2k
